# Data preparation

This script will output 4 files altogether:

1.Corpus_identification_con_met.csv: Modified conventional metaphor corpus from deliberate metaphor corpus.

2.sentence_overview.csv: Showing the distribution the sentence by random selection (percentage of train set and test set; examples; 10-fold).

3.sen4prompting.csv: sentences sorted in 10 groups used for labelling.

4.example4prompting.csv: example sentences and their word list with labels.

5.selected_permuted_prompts.csv: selection of 25 unique prompts from permutation of the whole prompt set for the experiment.

In [2]:
#Use the installation if there is pyreadstat, pandas or any other library missing
#pip install pyreadstat
#pip install pandas

  Obtaining dependency information for pyreadstat from https://files.pythonhosted.org/packages/d9/41/23d0ff53cf1fe687b1cd685bbf25785d6a5326ad4394a5855f8cdff33481/pyreadstat-1.2.4-cp38-cp38-win_amd64.whl.metadata
   ---------------------------------------- 2.4/2.4 MB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: orange3-imageanalytics 0.6.0 has a non-standard dependency specifier numpy>=1.16.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of orange3-imageanalytics or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version

In [4]:
#import needed libraries
import pandas as pd
import pyreadstat
import numpy as np
from itertools import permutations
import re

The conventional metaphor corpus will be adapted from the deliberate metaphor corpus, which is a spss file in sav format. The processing will include steps:

1.Make sure every word pair are connected with "+" and there is no space between words. For example, "I find+out the L-shaped desk". So in the later stage ChatGPT can just tokenize sentence by space (same tokenization way as that in the corpus).

2.The corpus consists of academic articles, converation texts, novels and News texts. Filter out others but News texts.

3.Reset the labels, non metaphor:0, conventional metaphor (non-deliberate metaphor):1, deliberate metaphor (activated conventional metaphor and novel metaphor, the original label):2.

4.Conbine words into sentence again.

5.Create sentence is based on docid.

In [7]:
# Make sure machine will always tokenize the text in the way we want
def join_multiple_words(word):
    # Ensure that word is a string
    word = str(word)
    # Use regular expressions to remove spaces around plus or minus signs
    word = re.sub(r"\s*\+\s*", "+", word)
    word = re.sub(r"\s*-\s*", "-", word)
    words = word.split()
    if len(words) > 1:
        return '+'.join(words)
    return word

# Read spss format deliberate metaphor corpus
df, meta = pyreadstat.read_sav('../data/corpus/Corpus_identification_del_met.sav')

# Filter the labelling of news texts 
df = df[df['reg'] == 'news']

# Replace the label of deliberate metaphor in DELMET column from 1to 2
df['DELMET'] = df['DELMET'].replace(1, 2)

# Find out the conventional metaphors(non-deliberate metaphors)
# When the 'type' column is 'met' and the 'DELMET' column is not 2, fill the cell corresponding to the 'DELMET' column with 1
df.loc[(df['type'] == 'met') & (df['DELMET'] != 2), 'DELMET'] = 1

# Find out the non metaphors (non-deliberate metaphors)
# When the 'type' column is not 'met', fill the cell corresponding to the 'DELMET' column with 1
df.loc[(df['type'] != 'met') & (df['DELMET'].isna()), 'DELMET'] = 0

# add two columns at the end 'context' and 'sentence_id'
df['context'] = None
df['sentence_id'] = None

# Used to save the current sentence
sentence_words = []
# Used to save the last docid of the current document
last_docid = None
# used to track the sentence order number of the current document
sentence_number = 1

# For loop DataFrame
for index, row in df.iterrows():
    # Apply new function to process words of 'word' column
    processed_word = join_multiple_words(row['word'])
    df.at[index, 'word'] = processed_word  # Make sure to update the DataFrame
    
    # Check if it is the firt word of a new document or the beginning of a new sentence
    if last_docid != row['docid'] or row['windex'] == 1:
        # If it is a new document, reset the sentence number
        if last_docid != row['docid']:
            sentence_number = 1
        # Combine the words from the previous sentence into a sentence and populate the context column
        if sentence_words:
            sentence = ' '.join(sentence_words)
            start_index = index - len(sentence_words)
            df.loc[start_index:index-1, 'context'] = sentence
            df.loc[start_index:index-1, 'sentence_id'] = f"{last_docid}_{sentence_number}"
            sentence_number += 1
            sentence_words = []  # Reset the word list of the current sentence
        last_docid = row['docid']
    
    # Add processed words to current sentence list
    sentence_words.append(processed_word)

# Populate the context column for the last sentence
if sentence_words:
    sentence = ' '.join(sentence_words)
    start_index = index - len(sentence_words) + 1
    df.loc[start_index:index, 'context'] = sentence
    df.loc[start_index:index, 'sentence_id'] = f"{last_docid}_{sentence_number}"

# Save the csv file
df.to_csv('../data/corpus/Corpus_identification_con_met.csv', index=False)

This step is about how to distribute the corpus (e.g. percentage of train set and test set; which sentences are used as examples in the prompt for maodel to learn):

1.Create a csv with unique sentences.

2.Add a column containing labels "train" or "test", and the percentage is train 10% vs test 90%.Randomly give sentences "train" or "test" labels.

3.Randomly select 10 examples respectively in train set and test set. Examples must not caontain deliberate metaphors.

4.Randomly seplit the train set into 10 groups (10-fold).

In [8]:
# load csv file
df = pd.read_csv('../data/corpus/Corpus_identification_con_met.csv')

# Select columns "sentence_id" and "context"
df = df[['sentence_id', 'context', 'DELMET']]

# label the sentences contain deliberate metaphor(s)
df['DEL'] = df.groupby(['sentence_id', 'context'])['DELMET'].transform(lambda x: 2 in x.values)

# remove duplicate sentences
df = df.drop_duplicates(subset=['context'])

#Delete Column DEL
df = df.drop(columns=['DELMET'])

# Set a random number seed to ensure replication
np.random.seed(1)

# Add a "split" column to randomly assign sentences to train and test
df['split'] = np.where(np.random.rand(len(df)) <= 0.1, 'train', 'test')

# Add an "examples" column and initialize all values to None
df['examples'] = None

# For train set and test set, 10 sentences are randomly repectively selected and marked as example, and the sentences marked as False in the DEL column are filtered out.
train_examples = df[(df['split'] == 'train') & (df['DEL'] == False)].sample(10, random_state=1).index
test_examples = df[(df['split'] == 'test') & (df['DEL'] == False)].sample(10, random_state=1).index
df.loc[train_examples, 'examples'] = 'example'
df.loc[test_examples, 'examples'] = 'example'

# df is the DataFrame
df['10_fold'] = None

# Randomly shuffle data
train_sentences = df[(df['split'] == 'train') & (df['examples'].isnull())].sample(frac=1, random_state=1)

# Randomly split the data set into 10 folds
folds = np.array_split(train_sentences.index, 10)
for fold_number, indices in enumerate(folds, 1):
    df.loc[indices, '10_fold'] = fold_number

# save the csv file
df.to_csv('../data/corpus/sentence_overview.csv', index=False)

Prepare a sentences in the way that can easily for loop in API: 10 groups of sentences in 10 cells in csv. In each cell, sentences are organized in the format of:

1.Sentence 1

2.Sentence 2

......

18.Sentence 18

In [9]:
# Replace 'your_file_path.csv' with the path to your CSV file
file_path = '../data/corpus/sentence_overview.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Function to create numbered sentences
def number_sentences(group):
    return "\n".join(f"{i+1}. {s}" for i, s in enumerate(group['context'].tolist()))

# Group by '10-fold' and apply the numbering function
grouped_df = df.groupby('10_fold').apply(number_sentences).reset_index(name='query_sentence')

# Save the result to a new CSV file
grouped_df.to_csv('../data/corpus/sen4prompting.csv', index=False)

Examples require sentence and corresponding word labelling.This step is about how to get them and store them in a csv.

In [10]:
# the data is loaded into the sentence_overview_df and corpus_df DataFrames.
# Replace 'path_to_sentence_overview_csv' and 'path_to_corpus_identification_csv' with the actual file paths
sentence_overview_df = pd.read_csv('../data/corpus/sentence_overview.csv')
corpus_df = pd.read_csv('../data/corpus/Corpus_identification_con_met.csv')

# Filter out the rows where 'examples' column has the word 'example'
filtered_sentence_overview_df = sentence_overview_df[sentence_overview_df['examples'] == 'example'][['examples', 'split', 'context','sentence_id']]

# Define a function to create a word list for the given context
def create_word_list(context):
    # Filter the corpus dataframe for the given context
    relevant_rows = corpus_df[corpus_df['context'] == context]
    # Format the word list, appending ':1' if DELMET is 1
    word_list = [f"{word}{':1' if delmet else ''}" for word, delmet in zip(relevant_rows['word'], relevant_rows['DELMET'])]
    # Join the list into a string with each word on a new line
    return "\n".join(word_list)

# Apply the function to the 'context' column to create the 'word_list' column
filtered_sentence_overview_df['word_list'] = filtered_sentence_overview_df['context'].apply(create_word_list)

# Save the csv file
filtered_sentence_overview_df.to_csv('../data/corpus/example4prompting.csv', index=False)

The following step is about preparing the prompts that will be used for the experiment:

1.Generating Prompt Permutations: we start by permuting prompt IDs.

2.Sampling Unique Sets: from the pool of all possible permutations, we draw a sample of 100 permutations. Each permutation consists of 5 prompt IDs, selected without repetition from the shuffled order.

3.Selecting Non-repetitive Prompts: ensure that the 25 total prompts selected (5 sets of 5 prompts) are all unique. 

4.Creating a CSV for the Experiment: the selected prompt IDs are then used to filter the original 25_Prompts_Cov_met.csv file. We extract the complete rows corresponding to the selected IDs and compile them into a new CSV file named selected_permuted_prompts.csv. This file will serve as the source of prompts for subsequent experimental procedures.

5.Final Verification: perform a final check to ensure that the selected_permuted_prompts.csv file contains the correct number of unique prompts, and that there are no duplications or omissions. 

In [5]:
# Load the csv file
prompts_df = pd.read_csv('../data/prompt_set/25_Prompts_Cov_met.csv')

# Get the 'prompt_id' column
prompt_ids = prompts_df['prompt_id'].values

# Define the number of permutations and the number of samples you want to take
P = 100  # Number of permutations
k = 5    # Number of prompt IDs in each sample

# Generate P permutations of the prompt IDs
permuted_ids = [np.random.permutation(prompt_ids) for _ in range(P)]

# Select the first k prompt IDs from each permutation
selected_permutations = [perm[:k] for perm in permuted_ids]

# Flatten the list of selected permutations to get all unique prompt IDs
unique_prompt_ids = np.unique(np.concatenate(selected_permutations))

# Filter the DataFrame for the selected prompt IDs
selected_prompts_df = prompts_df[prompts_df['prompt_id'].isin(unique_prompt_ids)]

# Save the csv file
selected_prompts_df.to_csv('../data/prompt_set/selected_permuted_prompts.csv', index=False)

# Print out the saved permutations
print(selected_prompts_df)

    prompt_id                                           Features   
0           1                              Bare task description  \
1           2                              Bare task description   
2           3                              Bare task description   
3           4                              Bare task description   
4           5                              Bare task description   
5           6                              Bare task description   
6           7  Bare task description + instruction steps + fe...   
7           8                            background information    
8           9        Word classification + bare task description   
9          10        Bare task description + word classification   
10         11                                          checklist   
11         12                            Research Assistant Role   
12         13                      Code-Like Parsing Instruction   
13         14                                 Co